## **Capstone MDS - ADABoost**

#### **_Integrantes:_**  

1. Evans Díaz   
1. Felipe Ojeda
1. Javier Cofre  
1. Luis Reinoso

In [1]:
# importar librerías
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from matplotlib.ticker import EngFormatter

In [2]:
# función cálculo R2, RMSE y MSE
def R2_RMSE(y_true, y_pred):
  # cálculo de R2 y RMSE
  r2 = r2_score(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse)
  
  return print('R2:',r2), print('RMSE:',rmse), print('MSE:', mse)

## **_1. Lectura (carga) de los Datos_**

In [3]:
# importamos archivo
df = pd.read_csv('../Data/df_final_pay.csv')

In [4]:
# miramos el dataframe
df.head()

,crew_E2,crew_E3,crew_E4,day_10,day_11,day_12,day_13,day_14,day_15,day_16,...,CAT 798-AC_des,KOM-CHQ_des,KOM.960E5_des,Kom.930E-4_des,Kom.930E-4SE_des,Kom.930EN_des,Kom.930ER_des,Kom.930ER-MT_des,Kom.980E5_des,Komatsu AHS_des
0,False,False,True,False,False,False,False,False,False,False,...,-0.21334,-0.575556,2.735792,0.191462,0.563663,2.277691,0.101727,-0.035749,-2.98167,-0.101592
1,True,False,False,False,False,False,False,False,False,False,...,-0.21334,-0.575556,2.735792,0.191462,0.607062,2.277691,0.101727,-0.035749,-2.98167,-0.101592
2,False,False,True,False,False,False,False,False,False,False,...,-0.21334,-0.575556,2.735792,0.191462,0.520264,2.277691,0.101727,-0.035749,-2.98167,-0.101592
3,True,False,False,False,False,False,False,False,False,False,...,-0.21334,-0.575556,2.735792,-1.756724,0.607062,2.277691,0.101727,-0.035749,-2.98167,-0.101592
4,False,False,True,False,False,False,False,False,False,False,...,-0.21334,-0.575556,2.735792,0.191462,0.563663,2.277691,0.101727,-0.035749,-2.98167,-0.101592


In [5]:
# información general df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 86 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   crew_E2               1565 non-null   bool   
 1   crew_E3               1565 non-null   bool   
 2   crew_E4               1565 non-null   bool   
 3   day_10                1565 non-null   bool   
 4   day_11                1565 non-null   bool   
 5   day_12                1565 non-null   bool   
 6   day_13                1565 non-null   bool   
 7   day_14                1565 non-null   bool   
 8   day_15                1565 non-null   bool   
 9   day_16                1565 non-null   bool   
 10  day_17                1565 non-null   bool   
 11  day_18                1565 non-null   bool   
 12  day_19                1565 non-null   bool   
 13  day_2                 1565 non-null   bool   
 14  day_20                1565 non-null   bool   
 15  day_21               

## **_2 Preparación del DataFrame_**

In [6]:
# transformamos las variables bool a categorical y le asignamos 0 o 1
for col in df.columns:
  if df[col].dtype == 'bool':
     df[col] = df[col].astype('category').cat.codes
     df[col] = df[col].astype('category') # para que quede como category y no como int8

In [7]:
print('Nombre Columnas: ',df.columns)
print('Cantidad Columnas: ',len(df.columns))

Nombre Columnas:  Index(['crew_E2', 'crew_E3', 'crew_E4', 'day_10', 'day_11', 'day_12', 'day_13',
       'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_2',
       'day_20', 'day_21', 'day_22', 'day_23', 'day_24', 'day_25', 'day_26',
       'day_27', 'day_28', 'day_29', 'day_3', 'day_30', 'day_31', 'day_4',
       'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'distance',
       'distance_down', 'distance_down_level', 'distance_up',
       'distance_up_level', 'lift_down', 'lift_down_level', 'lift_up',
       'lift_up_level', 'month_10', 'month_11', 'month_12', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'pay_original', 'shift_B', 'travel_time', 'year_2023',
       'year_2024', 'total_des', 'efectivo_des', 'mantencion_des',
       'mecanica_des', 'electrica_des', 'mant_excl_des', 'mec_excl_des',
       'elec_excl_des', 'som_excl_des', 'demora_des', 'r_prog_des',
       'r_noprg_des', 'waiting_des', 'queued_des', '

In [8]:
# cálculamos la desviaicón estándar y la media de payload
std_y = df['pay_original'].std()
mean_y = df['pay_original'].mean()

print('Desviación Estándar:',std_y)
print('Media:',mean_y)

Desviación Estándar: 19865.315566014815
Media: 142292.45186568802


## **_3 ADABoost_**

In [9]:
# definimos el tamaño del set de test
test_size = 0.25

In [10]:
# separamos los datos en X e y
X = df.drop('pay_original', axis=1)
y = df['pay_original']

# dividimos los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42) 

In [11]:
# Crear un regresor de árbol de decisión como modelo base, con un solo nodo de decisión (profundidad = 1)
base_estimator = DecisionTreeRegressor(max_depth=1)

In [12]:
# Inicializar el regresor AdaBoost
ada_boost_reg = AdaBoostRegressor(estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

In [13]:
# Entrenar el modelo AdaBoost
ada_boost_reg.fit(X_train, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1), random_state=42)

In [14]:
# Realizar predicciones con el modelo entrenado
predictions = ada_boost_reg.predict(X_test)

In [15]:
# Evaluar la precisión del modelo
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error of AdaBoost Regressor: {mse:.2f}')

Mean Squared Error of AdaBoost Regressor: 223941293.98


In [16]:
# resultado de R2, RMSE y MSE
R2_RMSE(y_test, predictions)

R2: 0.437960160091483
RMSE: 14964.668188105248
MSE: 223941293.9800892


(None, None, None)

## _4 Optimización del Modelo (I)_ 

### _4.1 Selección de Variables_

#### _4.1.1 Utilizando Lasso_

In [17]:
# buscamos alpha optimo con LassoCV
alpha_values = np.linspace(0.01, 300, 30000)

# Crea un objeto LassoCV
lasso_cv = LassoCV(alphas=alpha_values, cv=5)

# Ajusta el modelo a los datos de entrenamiento
lasso_cv.fit(X_train, y_train)

# Valor óptimo de alpha
best_alpha = lasso_cv.alpha_

# Evalúa el modelo en el conjunto de prueba
score = lasso_cv.score(X_test, y_test)

print(f"Mejor valor de alpha: {best_alpha}")
print(f"Puntuación en el conjunto de prueba: {score}")

NameError: name 'LassoCV' is not defined

In [ ]:
# Graficar MSE vs. Alpha
plt.figure(figsize=(10,6))
plt.plot(lasso_cv.alphas_, np.mean(lasso_cv.mse_path_, axis=1), '-o')
plt.xlabel('Alpha')
plt.ylabel('Mean MSE across CV folds')
plt.title('Path de Regularización')
plt.axvline(lasso_cv.alpha_, color='red', linestyle='--')

# Anotar el valor óptimo de alpha en el gráfico
plt.annotate(f'Optimal alpha: {lasso_cv.alpha_:.2f}, MSE: {np.min(np.mean(lasso_cv.mse_path_, axis=1)):.2f}',
             xy=(lasso_cv.alpha_, np.min(np.mean(lasso_cv.mse_path_, axis=1))),
             xycoords='data', xytext=(10,100), 
             textcoords='offset points',
             arrowprops=dict(arrowstyle="->", lw=1.5),
             fontsize=9)

plt.show()

In [ ]:
# Aplicar Lasso para la selección de características
lasso = Lasso(alpha=lasso_cv.alpha_)  
lasso.fit(X_train, y_train)

# Seleccionar las características que tienen coeficientes no nulos
selected_features = X_train.columns[lasso.coef_ != 0]

print("Características seleccionadas:")
print(selected_features)


In [ ]:
len(selected_features)

> Volvemos ajustar el modelo con las variables seleccionadas

In [ ]:
# separamos los datos en train y test
X_lasso= df[selected_features]
y_lasso = df['pay_original']

# dividimos los datos en train y test
X_train_lasso, X_test_lasso, y_train_lasso, y_test_lasso = train_test_split(X_lasso, y_lasso, test_size=test_size, random_state=42)

In [ ]:
# aplicamos la función de regresión lineal
lm_lasso = regresion_lineal(X_train_lasso, X_test_lasso, y_train_lasso, y_test_lasso)

In [ ]:
# sumario del modelo
ls = summary_regresion_lineal(X_train_lasso, y_train_lasso)

In [ ]:
test_predictions = lm_lasso.predict(X_test_lasso)
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test_lasso}).sort_index()
test_results

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(test_results['Test Predictions'])
plt.plot(test_results['Actuals'])

plt.title('Test Predictions vs Actuals (SimpleRNN)')
plt.legend(['Test Predictions', 'Actuals'])
plt.show()

In [ ]:
# Obtener los coeficientes del modelo Lasso
lasso_coef = lm_lasso.coef_

# Crear un DataFrame con las características y sus coeficientes
feature_importance = pd.DataFrame({'features': X_train_lasso.columns, 'importance': lasso_coef})

# Filtrar para mantener solo las características con coeficientes no nulos
feature_importance = feature_importance[feature_importance['importance'] != 0]

# Ordenar las características por importancia
feature_importance = feature_importance.sort_values(by='importance', ascending=True)

# Crear un gráfico de barras horizontales
plt.figure(figsize=(10, 8))
plt.barh(feature_importance['features'], feature_importance['importance'])
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.title('Importancia de las Características seleccionadas por Lasso')
plt.show()

### _4.2 Residuos_

#### _4.2.1 En Conjunto Entrenamiento_

> Nos quedamos con la selección de variables obtenidas en Lasso.

In [ ]:
# separamos los datos en X e y para las variables seleccionadas
X = df[X_train_lasso.columns]
y = df['pay_original']

# separamos los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
# iniciamos el modelo
lr = LinearRegression()

# ajustamos el modelo
lr.fit(X_train, y_train)

In [ ]:
# predicción conjunto entrenamiento y residuales
y_pred_train = lr.predict(X_train)
residual_train = y_train - y_pred_train

# ordenamos los datos
residual_train_order = residual_train.sort_index()
X_train_order = X_train.sort_index()

In [ ]:
# graficamos los residuales
plt.scatter(y_pred_train, residual_train)
plt.ylabel('Residuals')
plt.xlabel('Predicted values')

# graficamos la linea de tendencia
coefficients = np.polyfit(y_pred_train, residual_train, 1)
line = np.polyval(coefficients, y_pred_train)
plt.plot(y_pred_train, line, color='red', label='Tendencia cuadrática')

In [ ]:
# grafaica de los residuos vs el tiempo 
plt.figure(figsize=(80,10))
plt.plot(X_train_order.index, residual_train_order)

plt.show()

#### _4.2.2 En Conjunto Test_

In [ ]:
# predicción conjunto entrenamiento y residuales
y_pred_test = lr.predict(X_test)
residual_test = y_test - y_pred_test

# ordenamos los datos
residual_test_order = residual_test.sort_index()
X_test_order = X_test.sort_index()

In [ ]:
# gráfica de los residuos
residuals = y_test - y_pred_test
plt.scatter(y_pred_test, residuals)
plt.ylabel('Residuals')
plt.xlabel('Predicted values')

# graficamos la linea de tendencia
coefficients = np.polyfit(y_pred_test, residuals, 1)
line = np.polyval(coefficients, y_pred_test)
plt.plot(y_pred_test, line, color='red', label='Tendencia cuadrática')


In [ ]:
# ordenamos los residuos por indice de menor a mayor
residual_order = residuals.sort_index()
X_order = X_test.sort_index()

In [ ]:
# grafaica de los residuos vs el tiempo 
plt.figure(figsize=(80,10))
plt.plot(X_order.index, residual_order)

### _4.3 Heteroscedasticidad - Logaritmo Variable Objetivo_

> Quitamos un 'material_tonnage' que es 0 (revisar después del despeje de los outliers)

In [ ]:
# separamos los datos en train y test
X = df[X_train_lasso.columns]
y = np.log(df['pay_original'])

# separamos los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
# aplicamos la función de regresión lineal
lm_log = regresion_lineal(X_train, X_test, y_train, y_test)

In [ ]:
# calculamos el valor inverso de la transformación logaritmica para RMSE y MSE
RMSE = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(lm_log.predict(X_test))))
MSE = mean_squared_error(np.exp(y_test), np.exp(lm_log.predict(X_test)))
RMSE, MSE

In [ ]:
# sumario del modelo
summary_regresion_lineal(X_train, y_train)

## _5 Optimización del Modelo (II)_ 

In [ ]:
# generamos un nuevo dataframe con las variables seleccionadas
df_int = pd.concat([df[X_train_lasso.columns],df['pay_original']], axis=1).copy()
df_int

### _5.1 Interacción 'payload_des' por 'distance'_

In [ ]:
# creamos la nueva variable
df_int['pay_des_distance'] = df_int['payload_des'] * df_int['distance']
df_int['pay_des_distance']

In [ ]:
# separamos los datos en X e y
X_int1 = df_int.drop('pay_original', axis=1)
y_int1 = df_int['pay_original']

# dividimos los datos en train y test
X_train_int1, X_test_int1, y_train_int1, y_test_int1 = train_test_split(X_int1, y_int1, test_size=test_size, random_state=42)

In [ ]:
# aplicamos la función de regresión lineal
lm_int1 = regresion_lineal(X_train_int1, X_test_int1, y_train_int1, y_test_int1)

In [ ]:
# sumario del modelo
summary_regresion_lineal(X_train_int1, y_train_int1)

### _5.2 Interacción 'payload_des' por 'expected_time_des'_

In [ ]:
# creamos la nueva variable
df_int['pay_des_exp_time_des'] = df_int['payload_des'] * df_int['expected_time_des']
df_int['pay_des_exp_time_des']

In [ ]:
# separamos los datos en X e y
X_int2 = df_int.drop('pay_original', axis=1)
y_int2 = df_int['pay_original']

# dividimos los datos en train y test
X_train_int2, X_test_int2, y_train_int2, y_test_int2 = train_test_split(X_int2, y_int2, test_size=test_size, random_state=42)

In [ ]:
# aplicamos la función de regresión lineal
lm_int2 = regresion_lineal(X_train_int2, X_test_int2, y_train_int2, y_test_int2)

In [ ]:
# sumario del modelo
summary_regresion_lineal(X_train_int2, y_train_int2)

### _5.3 Interacción 'payload_des' por 'CAT 797-F_des'_

In [ ]:
# creamos la nueva variable
df_int['pay_des_CAT_des'] = df_int['payload_des'] * df_int['CAT 797-F_des']
df_int['pay_des_CAT_des']

In [ ]:
# separamos los datos en X e y
X_int3 = df_int.drop('pay_original', axis=1)
y_int3 = df_int['pay_original']

# dividimos los datos en train y test
X_train_int3, X_test_int3, y_train_int3, y_test_int3 = train_test_split(X_int3, y_int3, test_size=test_size, random_state=42)

In [ ]:
# aplicamos la función de regresión lineal
lm_int3 = regresion_lineal(X_train_int3, X_test_int3, y_train_int3, y_test_int3)

In [ ]:
# sumario del modelo
summary_regresion_lineal(X_train_int3, y_train_int3)

In [ ]:
# graficamos 'payload_original' vs 'lift_down'
sns.scatterplot(x='pay_des_distance', y='pay_original', data=df_int, alpha=0.5, color='gray')

sns.despine()
plt.title('Payload Original vs pay_des_distance') 
plt.gca().yaxis.set_major_formatter(EngFormatter())
plt.show()